In [1]:
#Loading the csv file and get data
import pandas as pd
data = pd.DataFrame()
path_datas = '../Datasets/Original/TSLA.csv'
trade_data = pd.read_csv(path_datas ,sep = ',',)
trade_data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900
5,2010-07-07,16.400000,16.629999,14.980000,15.800000,15.800000,6921700
6,2010-07-08,16.139999,17.520000,15.570000,17.459999,17.459999,7711400
7,2010-07-09,17.580000,17.900000,16.549999,17.400000,17.400000,4050600
8,2010-07-12,17.950001,18.070000,17.000000,17.049999,17.049999,2202500
9,2010-07-13,17.389999,18.639999,16.900000,18.139999,18.139999,2680100


In [2]:
columns_names = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Kopie des DataFrames erstellen
modified_trade_data = trade_data.copy()
t_previous_days = 2

for attribute in columns_names:
    for i in range(1, t_previous_days + 1):
        deviation_col_name = f'deviation_{attribute}_{i}'
        modified_trade_data[deviation_col_name] = modified_trade_data[attribute] - modified_trade_data[attribute].shift(i)
        
        average_deviation_col_name = f'average_deviation_{attribute}'
        modified_trade_data[average_deviation_col_name] = modified_trade_data[[f'deviation_{attribute}_{j}' for j in range(1, i + 1)]].mean(axis=1)

    positive_deviation_col_name = f'positive_deviations_{attribute}'
    modified_trade_data[positive_deviation_col_name] = 0
    for i in range(1, t_previous_days + 1):
        modified_trade_data[positive_deviation_col_name] += (modified_trade_data[f'deviation_{attribute}_{i}'] > 0).astype(int)

    negative_deviation_col_name = f'negative_deviations_{attribute}'
    modified_trade_data[negative_deviation_col_name] = 0
    for i in range(1, t_previous_days + 1):
        modified_trade_data[negative_deviation_col_name] += (modified_trade_data[f'deviation_{attribute}_{i}'] < 0).astype(int)

for i in range(1, t_previous_days + 1):
    diff_col_name_open_close = f'difference_open_close_{i}'
    modified_trade_data[diff_col_name_open_close] = modified_trade_data['Open'] - modified_trade_data['Close'].shift(i)
    

    diff_col_name = f'difference_high_low_{i}'
    modified_trade_data[diff_col_name] = modified_trade_data[attribute] - modified_trade_data[attribute].shift(i)

average_diff_col_name = f'average_difference_high_low'
modified_trade_data[average_diff_col_name] = modified_trade_data[diff_col_name].rolling(window=t_previous_days).mean()
average_diff_col_name_open_close = f'average_difference_open_close'
modified_trade_data[average_diff_col_name_open_close] = modified_trade_data[diff_col_name_open_close].rolling(window=t_previous_days).mean()


modified_trade_data.dropna(inplace=True)
modified_trade_data.head()


,Date,Open,High,Low,Close,Adj Close,Volume,deviation_Open_1,average_deviation_Open,deviation_Open_2,...,average_deviation_Volume,deviation_Volume_2,positive_deviations_Volume,negative_deviations_Volume,difference_open_close_1,difference_high_low_1,difference_open_close_2,difference_high_low_2,average_difference_high_low,average_difference_open_close
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800,-2.000000,-2.395001,-2.790001,...,-7563150.0,-12047300.0,0,2,1.040001,-3079000.0,-0.830000,-12047300.0,-11297400.0,0.140001
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900,-3.000000,-4.000000,-5.000000,...,187600.0,-1351900.0,1,1,0.799999,1727100.0,-1.959999,-1351900.0,-6699600.0,-1.394999
5,2010-07-07,16.400000,16.629999,14.980000,15.800000,15.800000,6921700,-3.600000,-5.100000,-6.600000,...,918350.0,1781900.0,2,0,0.289999,54800.0,-2.800001,1781900.0,215000.0,-2.380000
6,2010-07-08,16.139999,17.520000,15.570000,17.459999,17.459999,7711400,-0.260001,-2.060001,-3.860001,...,817100.0,844500.0,2,0,0.339999,789700.0,0.029998,844500.0,1313200.0,-1.385002
7,2010-07-09,17.580000,17.900000,16.549999,17.400000,17.400000,4050600,1.440001,1.310000,1.180000,...,-3265950.0,-2871100.0,0,2,0.120001,-3660800.0,1.780000,-2871100.0,-1013300.0,0.904999


In [3]:
# Annahme: trade_data ist dein DataFrame
# Verwende die Funktion `columns` des DataFrames, um alle Spaltennamen abzurufen
column_names = modified_trade_data.columns.tolist()

# Ausgabe der Spaltennamen
print("Spaltennamen des DataFrames:")
for column in column_names:
    print(column)


Spaltennamen des DataFrames:
Date
Open
High
Low
Close
Adj Close
Volume
deviation_Open_1
average_deviation_Open
deviation_Open_2
positive_deviations_Open
negative_deviations_Open
deviation_High_1
average_deviation_High
deviation_High_2
positive_deviations_High
negative_deviations_High
deviation_Low_1
average_deviation_Low
deviation_Low_2
positive_deviations_Low
negative_deviations_Low
deviation_Close_1
average_deviation_Close
deviation_Close_2
positive_deviations_Close
negative_deviations_Close
deviation_Adj Close_1
average_deviation_Adj Close
deviation_Adj Close_2
positive_deviations_Adj Close
negative_deviations_Adj Close
deviation_Volume_1
average_deviation_Volume
deviation_Volume_2
positive_deviations_Volume
negative_deviations_Volume
difference_open_close_1
difference_high_low_1
difference_open_close_2
difference_high_low_2
average_difference_high_low
average_difference_open_close
